In [219]:
from __future__ import division
import pandas as pd,re,numpy as np
from functools import reduce
from pyomo.environ import *
import pyomo.environ as pyo
import itertools as it

In [220]:
coef_df = pd.read_csv("coefficient_matrix.csv")

annual_df = pd.read_csv("AnnualOptimizerInputData.csv")

quarter_df = pd.read_csv("LastQuarterData.csv")

In [221]:
Tot_Prod = coef_df["Modelled_Product"].nunique()

Tot_Week = coef_df["wk"].nunique()

In [222]:
Min_EDLP_Events = list(annual_df["EDLP_Events_Min"])

Max_EDLP_Events = list(annual_df["EDLP_Events_Max"])

Min_TPR_Events = list(annual_df["TPR_Events_Min"])

Max_TPR_Events = list(annual_df["TPR_Events_Max"])

Ov_Perc_Limit, TPR_Perc_Limit, EDLP_Perc_Limit = 10, 20, 20

In [223]:
# Base_Price_Products

lis = [1.99, 3.05, 3.53, 3.01, 3.45, 0.99]

# Base_Price = [[i]*Tot_Week for i in quarter_df["Base Price"]]

Base_Price = [[i]*Tot_Week for i in lis]

print(Base_Price)



[[1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99], [3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05], [3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53, 3.53], [3.01, 3.01, 3.01, 3.01, 3.01, 3.01, 3.01, 3.01, 3.01, 3.

In [224]:
Target_EDLP_Spend = [ 3137.737996, 18848.59057, 5119.820028, 8578.147936, 19798.3005, 0]

Target_TPR_Spend = [0, 0, 691.204, 1052.194, 3304.293,0]

Target_Trade_Spend = list(map(lambda x,y:x+y,Target_EDLP_Spend,Target_TPR_Spend))

Target_EDLP_Spend, Target_TPR_Spend, Target_Trade_Spend

([3137.737996, 18848.59057, 5119.820028, 8578.147936, 19798.3005, 0],
 [0, 0, 691.204, 1052.194, 3304.293, 0],
 [3137.737996,
  18848.59057,
  5811.024028,
  9630.341935999999,
  23102.593500000003,
  0])

In [225]:
con = [i for i in coef_df["Constant"]]

Intercepts = [con[j:j+Tot_Prod] for j in range(0,len(con),Tot_Prod)]

# print(Intercepts)

In [226]:
EDLP_Coef = np.array(coef_df[[i for i in coef_df.columns if i.count("EDLP")==1 ]])
# print(EDLP_Coef)

In [227]:
TPR_Coef = np.array(coef_df[[i for i in coef_df.columns if i.count("TPR")==1 ]])
# print(TPR_Coef)

In [228]:
# Available EDLP Interactions pairs

EDLP = [re.findall("[0-9]",i) for i in coef_df.columns if i.count("EDLP")>1 ] 

EDLP_Interactions = []

for i in EDLP:

    temp=[]
    
    for j in i:
        
        temp.append(int(j)-1)
        
    EDLP_Interactions.append(temp)

EDLP_Interactions

[[0, 5]]

In [229]:
# Available TPR Interactions pairs

TPR = [re.findall("[0-9]",i) for i in coef_df.columns if i.count("TPR")>1 ] 

TPR_Interactions = []

for i in TPR:

        temp=[]
        
        for j in i:
        
            temp.append(int(j)-1)
        
        TPR_Interactions.append(temp)

# TPR_Interactions

In [230]:
# EDLP_Interaction_Coef_Values

EDLP_Int_Coef_Values = {}

for col in coef_df.columns:
    
    if col.count("EDLP")>1 :
        
        Pair_name ="_".join([str(int(i)-1) for i in re.findall("[0-9]",col)])
                
        EDLP_Int_Coef_Values[Pair_name] = list(coef_df[col])

In [231]:
# TPR_Interaction_Coef_Values

TPR_Int_Coef_Values = {}

for col in coef_df.columns:
    
    if col.count("TPR")>1 :
        
        Pair_name ="_".join([str(int(i)-1) for i in re.findall("[0-9]",col)])
                
        TPR_Int_Coef_Values[Pair_name] = list(coef_df[col])

In [232]:
def Retailer_Unit_Sales_Fn(Model,P,W):
    
    Self = exp( (log(Model.EDLP[P,W] * Base_Price[P][W]) * EDLP_Coef[P][P]) * Model.FLAG[P,W]
                     
                     + (log(Base_Price[P][W]) * EDLP_Coef[P][P] + Model.TPR[P,W] * TPR_Coef[P][P]) * (1 - Model.FLAG[P,W])
                     
                     + Intercepts[W][P])
    
    Comp = Competitor_Unit_Effect_Fn(Model,P,W)
    
    EDLP_Inter_Val = EDLP_Interaction_Unit_Effect_Fn(Model,P,W)
    
    TPR_Inter_Val = TPR_Interaction_Unit_Effect_Fn(Model,P,W)    
    
    Unit_Sales = Self * Comp * EDLP_Inter_Val * TPR_Inter_Val
    
    
    return Unit_Sales

In [233]:
def Competitor_Unit_Effect_Fn(Model,Cur_Ret,Wk):
    
    Comp_Retailers_Unit_Sales = [exp((log(Model.EDLP[i, Wk] * Base_Price[i][Wk]) * EDLP_Coef[Cur_Ret][i]) * Model.FLAG[i, Wk] +
                                     
                                     (log(Base_Price[i][Wk]) * EDLP_Coef[Cur_Ret][i] + Model.TPR[i, Wk] * TPR_Coef[Cur_Ret][i]) *
                                     
                                     (1 - Model.FLAG[i, Wk])) for i in Model.PPG_index if i != Cur_Ret]
    
    
    return reduce(lambda x,y:x*y, Comp_Retailers_Unit_Sales,1)

In [234]:
def EDLP_Interaction_Unit_Effect_Fn(Model,P,Wk):
    
    EDLP_Interaction_Unit_Effect = [1]
    
    for Inter in EDLP_Interactions:
    
            if P in Inter:
                
                Pr_ind1, Pr_ind2 = Inter
                
                Comb = str(Pr_ind1) + "_" + str(Pr_ind2)
                
                Val = exp((Base_Price[Pr_ind1][Wk] * Base_Price[Pr_ind2][Wk] * EDLP_Int_Coef_Values[Comb][P] *
                       
                        (1 - Model.FLAG[Pr_ind1,Wk]) * (1 - Model.FLAG[Pr_ind2,Wk])) +
                       
                       (Base_Price[Pr_ind1][Wk] * log(Model.EDLP[Pr_ind2,Wk] * Base_Price[Pr_ind2][Wk]) *
                        
                        EDLP_Int_Coef_Values[Comb][P] * (1 - Model.FLAG[Pr_ind1,Wk]) * (Model.FLAG[Pr_ind2,Wk]) ) +
                       
                       (log(Base_Price[Pr_ind1][Wk] * Model.EDLP[Pr_ind1,Wk]) * Base_Price[Pr_ind2][Wk] * 
                        
                        EDLP_Int_Coef_Values[Comb][P] * (Model.FLAG[Pr_ind1,Wk]) * (1 - Model.FLAG[Pr_ind2,Wk])) +
                       
                       (log(Base_Price[Pr_ind1][Wk] * Model.EDLP[Pr_ind1,Wk]) * log(Base_Price[Pr_ind2][Wk] * 
                        
                        Model.EDLP[Pr_ind2,Wk]) * EDLP_Int_Coef_Values[Comb][P] * 
                       
                        (Model.FLAG[Pr_ind1,Wk]) * (Model.FLAG[Pr_ind2,Wk])) )
                        
                EDLP_Interaction_Unit_Effect.append(Val)
    
    return reduce(lambda x,y: x*y, EDLP_Interaction_Unit_Effect)

In [235]:
def TPR_Interaction_Unit_Effect_Fn(Model,P,Wk):
    
    TPR_Interaction_Unit_Effect = [1]
    
    for Inter in TPR_Interactions:
    
            if P in Inter:
                
                Pr_ind1, Pr_ind2 = Inter
                
                Comb = str(Pr_ind1) + "_" + str(Pr_ind2)
                
                Val = exp(( Model.TPR[Pr_ind1,Wk] * Model.TPR[Pr_ind2,Wk] *
                           
                           TPR_Int_Coef_Values[Comb][P] * (1 - Model.FLAG[Pr_ind1,Wk]) * (1 - Model.FLAG[Pr_ind2,Wk])) ) 
                          
                TPR_Interaction_Unit_Effect.append(Val)
    
    return reduce(lambda x,y: x*y, TPR_Interaction_Unit_Effect)

In [236]:
def Retailer_Price_Fn(Model,P,W):
    
    Price = ((Base_Price[P][W] * (1 - Model.TPR[P,W]) * (1 - Model.FLAG[P,W])) + 
             
             (Model.EDLP[P,W] * Base_Price[P][W] * Model.FLAG[P,W]) )
    
    return Price

In [237]:
def Dollar_Sales_Fn(Model):
    
#     print(( [ Retailer_Unit_Sales_Fn(Model,P,W) * Retailer_Price_Fn(Model,P,W) for W in Model.Wk_index
                 
#                  for P in Model.PPG_index] ))
    
    return sum( [ Retailer_Unit_Sales_Fn(Model,P,W) * Retailer_Price_Fn(Model,P,W) for W in Model.Wk_index
                 
                 for P in Model.PPG_index] )

In [238]:
def FLAG_Util_Fn(Model, Cur_Ret):
 
    return sum([Model.FLAG[Cur_Ret, Wk] * (1 - Model.FLAG[Cur_Ret, Wk]) for Wk in range(Tot_Week) ]) == 0

In [239]:
def EDLP_Event_Fn(Model,P):
    
    return pyo.inequality(Min_EDLP_Events[P], sum([Model.FLAG[P,W] for W in Model.Wk_index]), Max_EDLP_Events[P])

In [240]:
def TPR_Event_Fn(Model,P):
    
    return pyo.inequality(Min_TPR_Events[P],Tot_Week- sum([Model.FLAG[P,W]  for W in Model.Wk_index]), Max_TPR_Events[P])

In [241]:
def Total_Trade_Spent_Bnd_Fn(Model,Cur_Ret):
    
    Val = sum( [ (Base_Price[Cur_Ret][Wk] - Retailer_Price_Fn(Model,Cur_Ret,Wk)) * Retailer_Unit_Sales_Fn(Model,Cur_Ret,Wk)
               
                 for Wk in Model.Wk_index]) 
    
    return  pyo.inequality(Target_Trade_Spend[Cur_Ret] * (1 - Ov_Perc_Limit/100), Val, Target_Trade_Spend[Cur_Ret] *
                           
                           (1 + Ov_Perc_Limit/100) )

In [242]:
def EDLP_Trade_Spent_Bnd_Fn(Model,Cur_Ret):
    
    Val = sum( [ (Base_Price[Cur_Ret][Wk] - Retailer_Price_Fn(Model,Cur_Ret,Wk)) * Retailer_Unit_Sales_Fn(Model,Cur_Ret,Wk) 
                 
               *  (Model.FLAG[Cur_Ret,Wk]) for Wk in Model.Wk_index])
    
    return  pyo.inequality(Target_EDLP_Spend[Cur_Ret] * (1 - EDLP_Perc_Limit/100), Val, 
                           
                           Target_EDLP_Spend[Cur_Ret] * (1 + EDLP_Perc_Limit/100) )

In [243]:
def TPR_Trade_Spent_Bnd_Fn(Model,Cur_Ret):
    
    Val = sum( [ (Base_Price[Cur_Ret][Wk] - Retailer_Price_Fn(Model,Cur_Ret,Wk)) * Retailer_Unit_Sales_Fn(Model,Cur_Ret,Wk) 
                 
               *  (1 - Model.FLAG[Cur_Ret,Wk]) for Wk in range(Tot_Week)])
    
    return pyo.inequality(Target_TPR_Spend[Cur_Ret] * (1 - TPR_Perc_Limit/100), Val, 
                          
                          Target_TPR_Spend[Cur_Ret] * (1 + TPR_Perc_Limit/100))

In [244]:
def Overall_Total_Trade_Spent_Fn(Model):
    
    return sum([sum([(Base_Price[Cur_Ret][Wk] - Retailer_Price_Fn(Model, Cur_Ret, Wk)) * Retailer_Unit_Sales_Fn(Model, Cur_Ret, Wk) 
                    
                     for Wk in range(Tot_Week)]) for Cur_Ret in range(Tot_Prod)]) == sum(Target_Trade_Spend)


In [249]:
solution = pd.read_csv("Weekly_optimiser_2_results.csv",index_col=0)
solution["EDLP_Perc"] =solution["EDLP_Price"]/solution["BasePrice"] 
solution.sort_values(by=["Product","week"],inplace=True)

edlp = list(solution["EDLP_Perc"])
edlp_init = np.array([edlp[j:j+Tot_Week] for j in range(0,len(solution.index),Tot_Week) ])

tpr = list(solution["TPR_discount"])
tpr_init = np.array([tpr[j:j+Tot_Week] for j in range(0,len(solution.index),Tot_Week) ])

flag = list(solution["Event_flag"])
flag_init = np.array([flag[j:j+Tot_Week] for j in range(0,len(solution.index),Tot_Week) ])

In [255]:
def Create_Model(Flag_Init,EDLP_Init,TPR_Init):
    
    print("Flag Initialization Value = ",Flag_Init)

    print("EDLP Initialization Value = ",EDLP_Init)

    print("TPR Initialization Value = ",TPR_Init)

    def Flag_Initialize(Model, *Index):
        
        return Flag_Init[Index]
    
    def EDLP_Initialize(Model, *Index):
        
        return EDLP_Init[Index]
    
    def TPR_Initialize(Model, *Index):
        
        return TPR_Init[Index]
    
    Model = ConcreteModel(name='Spend_Optim')
    
    Model.Weeks = Param(initialize=Tot_Week, domain=PositiveIntegers)

    Model.PPGs = Param(initialize=Tot_Prod, domain=PositiveIntegers)

    Model.Wk_index = RangeSet(0, Model.Weeks - 1)
    
    Model.PPG_index = RangeSet(0, Model.PPGs - 1)
   
    Model.EDLP = Var(Model.PPG_index, Model.Wk_index, initialize=EDLP_Initialize, domain=NonNegativeReals)
    
    Model.TPR = Var(Model.PPG_index, Model.Wk_index, initialize=TPR_Initialize, domain=NonNegativeReals)

    Model.FLAG = Var(Model.PPG_index, Model.Wk_index, initialize=Flag_Initialize, domain=Binary)

    Model.Obj = Objective(rule = Dollar_Sales_Fn, sense=maximize)
    
    Model.Tot_Spent_Bnd = Constraint(Model.PPG_index, rule=Total_Trade_Spent_Bnd_Fn)
    
    Model.EDLP_Bnd = Constraint(Model.PPG_index, rule=EDLP_Trade_Spent_Bnd_Fn)
    
    Model.TPR_Bnd = Constraint(Model.PPG_index,rule=TPR_Trade_Spent_Bnd_Fn)
    
#     Model.FLAG_Util_c = Constraint(Model.PPG_index, rule=FLAG_Util_Fn)
    
    Model.TPR_Event = Constraint(Model.PPG_index,rule=TPR_Event_Fn)
    
    Model.EDLP_Event = Constraint(Model.PPG_index,rule=EDLP_Event_Fn)
    
#     Model.Overall = Constraint(rule=Overall_Total_Trade_Spent_Fn)

    return Model



In [256]:
def call_solver(Flag_Init,EDLP_Init,TPR_Init, PPGs, Wks, name='bonmin'):
    
    Model = Create_Model(Flag_Init,EDLP_Init,TPR_Init)
    
    Opt = SolverFactory(name)
    
    Result = Opt.solve(Model)
    
    print(pyomo.environ.value(Model.Obj))
    
    print([pyo.value(Model.EDLP[P,Wk]) for P in range(PPGs) for Wk in range(Wks)])
    
    print([pyo.value(Model.TPR[P, Wk]) for P in range(PPGs) for Wk in range(Wks)])
    
    print([pyo.value(Model.FLAG[P, Wk]) for P in range(PPGs) for Wk in range(Wks)])
    
    print([(P, Wk) for P in range(PPGs) for Wk in range(Wks)])
    
    Model.display()


In [257]:
call_solver(flag_init,edlp_init,tpr_init,PPGs=6, Wks=52, name='bonmin')

Flag Initialization Value =  [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
EDLP Initialization Value =  [[0.99497487 0.99497487 0.99497487 0.99497487 0.99497487 0.99497487
  0.99497487 0.99497487 0.99497487 0.99497487 0.99497487 0.99497487
  0.99497487 0.99497487 0.99497487 0.99497487 0.99497487 0.99497487
  0.99497487 0.99497487 0.99497487 0.99497487 0.99497487 0.99497487
  0.99497487 0.

        Key : Lower              : Body               : Upper
          0 :       2510.1903968 :  3137.737996124043 : 3765.2855951999995
          1 : 15078.872456000001 : 18848.590569294192 :       22618.308684
          2 : 4095.8560224000003 :  5119.820027651015 :       6143.7840336
          3 :       6862.5183488 :   8578.14793628443 : 10293.777523199999
          4 : 15838.640400000002 : 19798.300500139387 : 23757.960600000002
          5 :                0.0 :                0.0 :                0.0
    TPR_Bnd : Size=6
        Key : Lower             : Body               : Upper
          0 :               0.0 :                0.0 :                0.0
          1 :               0.0 :                0.0 :                0.0
          2 :          552.9632 :  691.2039942654898 :  829.4447999999999
          3 : 841.7552000000001 : 1052.1935703859115 : 1262.6327999999999
          4 :         2643.4344 :  3304.292711885247 :          3965.1516
          5 :               0.0 :   